# Topology Summary Generator

This notebook provides tools to generate and inject descriptive summaries into topology YAML files.

## Features:
- 📝 Generate detailed topology summaries
- 🎨 Create ASCII art diagrams
- 💾 Inject summaries into YAML files as comment headers
- 🔒 Automatic backup creation
- 🔄 Works with both model-based and dict-based topologies

## Setup

In [ ]:
# Import Setup
import sys
from pathlib import Path

# Add parent directory to Python path to import modules
repo_root = Path.cwd().parent
sys.path.insert(0, str(repo_root))

print(f"✅ Python path configured")
print(f"   Repository root: {repo_root}")

In [ ]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path

# Import modules
from slice_utils_models import load_topology_from_yaml_file
import slice_topology_viewer as viewer

print("✅ Modules imported successfully")

In [ ]:
# Define YAML directory
YAML_DIR = repo_root / "model"

# Updated configuration cell (replaces the old one)
# Topology YAML file path (now in yaml/ directory)
yaml_file = YAML_DIR / "../model/_slice_topology.yaml"

print(f"✅ YAML directory: {YAML_DIR}")

# Check if file exists
if yaml_file.exists():
    print(f"✅ Found topology file: {yaml_file.name}")
else:
    print(f"❌ File not found: {yaml_file}")
    print(f"\nAvailable YAML files in {YAML_DIR}:")
    for f in YAML_DIR.glob("*.yaml"):
        print(f"  - {f.name}")

## Configuration

Specify your topology YAML file:

In [ ]:
# Topology YAML file path
yaml_file = "../model/_slice_topology.yaml"

# Check if file exists
if Path(yaml_file).exists():
    print(f"✅ Found topology file: {yaml_file}")
else:
    print(f"❌ File not found: {yaml_file}")
    print("\nPlease update the yaml_file variable with the correct path.")

## Load Topology

Load and validate the topology:

In [ ]:
try:
    topology = load_topology_from_yaml_file(yaml_file)
    print("✅ Topology loaded and validated successfully!")
    print(f"\n📊 Statistics:")
    print(f"   • Nodes: {len(topology.site_topology_nodes.nodes)}")
    print(f"   • Networks: {len(topology.site_topology_networks.networks)}")
    
    # List nodes
    print(f"\n🖥️  Nodes:")
    for node in topology.site_topology_nodes.iter_nodes():
        print(f"   • {node.hostname} ({node.site})")
    
    # List networks
    print(f"\n🌐 Networks:")
    for network in topology.site_topology_networks.iter_networks():
        print(f"   • {network.name} ({network.subnet})")
        
except Exception as e:
    print(f"❌ Error loading topology: {e}")
    import traceback
    traceback.print_exc()

---

# Option 1: Command-Line Tool

Use the standalone script for command-line operations.

## Basic Usage

Run these commands in your terminal:

In [ ]:
# Display command-line usage examples
print("""
Command-Line Usage Examples:
═══════════════════════════════════════════════════════════════

1. Inject summary into YAML file (creates backup automatically):
   python tool_topology_summary_generator.py _slice_topology.yaml

2. Generate summary to separate file:
   python tool_topology_summary_generator.py _slice_topology.yaml --output summary.txt

3. Without ASCII diagram:
   python tool_topology_summary_generator.py _slice_topology.yaml --no-ascii

4. No backup:
   python tool_topology_summary_generator.py _slice_topology.yaml --no-backup

5. Preserve existing comments:
   python tool_topology_summary_generator.py _slice_topology.yaml --preserve-comments

6. Verbose output:
   python tool_topology_summary_generator.py _slice_topology.yaml -v

7. Get help:
   python tool_topology_summary_generator.py --help

""")

## Run from Notebook

You can also execute the command-line tool from within the notebook:

In [ ]:
# Example: Run the command-line tool from notebook
!python tool_topology_summary_generator.py {yaml_file} --help

In [ ]:
# Generate summary to separate file (non-destructive)
output_file = "topology_summary.txt"
!python tool_topology_summary_generator.py {yaml_file} --output {output_file}

# Display the generated summary
if Path(output_file).exists():
    print("\n" + "="*70)
    print("Generated Summary:")
    print("="*70)
    print(Path(output_file).read_text())

---

# Option 2: Programmatic API

Use Python functions for more control and integration with your workflow.

## 2.1 Preview Summary (Non-Destructive)

Generate and display the summary without modifying any files:

In [ ]:
# Generate summary text
summary = viewer.generate_yaml_summary(topology, include_ascii=True)

print("="*70)
print("GENERATED SUMMARY PREVIEW")
print("="*70)
print(summary)

## 2.2 Generate Summary Without ASCII Diagram

In [ ]:
# Generate summary without ASCII diagram
summary_no_ascii = viewer.generate_yaml_summary(topology, include_ascii=False)

print("="*70)
print("SUMMARY WITHOUT ASCII DIAGRAM")
print("="*70)
print(summary_no_ascii)

## 2.3 Save Summary to Separate File

In [ ]:
# Save summary to a separate text file
output_file = "topology_summary_detailed.txt"

summary = viewer.generate_yaml_summary(topology, include_ascii=True)
Path(output_file).write_text(summary)

print(f"✅ Summary saved to: {output_file}")
print(f"   File size: {Path(output_file).stat().st_size} bytes")

## 2.4 Inject Summary into YAML File

⚠️ **Warning**: This will modify your YAML file (backup is created automatically)

In [ ]:
# Configuration
create_backup = True          # Create .bak file before modifying
include_ascii_diagram = True  # Include ASCII network diagram

print(f"Configuration:")
print(f"  • Target file: {yaml_file}")
print(f"  • Create backup: {create_backup}")
print(f"  • Include ASCII: {include_ascii_diagram}")
print("\nℹ️  A backup file (.bak) will be created before modification.")
print("\n⚠️  Run the next cell to inject the summary into the YAML file.")

In [ ]:
# Inject summary into YAML file
try:
    viewer.inject_summary_into_yaml_file(
        yaml_file,
        topology,
        include_ascii=include_ascii_diagram,
        backup=create_backup
    )
    
    print("\n✅ Summary successfully injected into YAML file!")
    
    if create_backup:
        backup_file = Path(yaml_file).with_suffix(Path(yaml_file).suffix + '.bak')
        if backup_file.exists():
            print(f"\n💾 Backup saved to: {backup_file}")
    
    # Display first few lines of updated file
    print("\n📄 First 30 lines of updated file:")
    print("="*70)
    lines = Path(yaml_file).read_text().split('\n')[:30]
    print('\n'.join(lines))
    print("...")
    
except Exception as e:
    print(f"❌ Error injecting summary: {e}")
    import traceback
    traceback.print_exc()

## 2.5 Restore from Backup (If Needed)

In [ ]:
# Restore original file from backup
backup_file = Path(yaml_file).with_suffix(Path(yaml_file).suffix + '.bak')

if backup_file.exists():
    print(f"Found backup: {backup_file}")
    print("\n⚠️  Run the next cell to restore from backup.")
else:
    print(f"❌ No backup file found: {backup_file}")

In [ ]:
# Uncomment and run to restore from backup
# backup_file = Path(yaml_file).with_suffix(Path(yaml_file).suffix + '.bak')
# if backup_file.exists():
#     backup_content = backup_file.read_text()
#     Path(yaml_file).write_text(backup_content)
#     print(f"✅ File restored from backup: {yaml_file}")
# else:
#     print(f"❌ Backup file not found")

---

# Additional Tools

## Compare Original vs Updated File

In [ ]:
backup_file = Path(yaml_file).with_suffix(Path(yaml_file).suffix + '.bak')

if backup_file.exists():
    original_size = backup_file.stat().st_size
    updated_size = Path(yaml_file).stat().st_size
    
    print("📊 File Comparison:")
    print(f"   Original: {original_size:,} bytes")
    print(f"   Updated:  {updated_size:,} bytes")
    print(f"   Difference: +{updated_size - original_size:,} bytes")
    
    # Count comment lines
    original_lines = backup_file.read_text().split('\n')
    updated_lines = Path(yaml_file).read_text().split('\n')
    
    original_comments = sum(1 for line in original_lines if line.strip().startswith('#'))
    updated_comments = sum(1 for line in updated_lines if line.strip().startswith('#'))
    
    print(f"\n   Original comment lines: {original_comments}")
    print(f"   Updated comment lines:  {updated_comments}")
    print(f"   New comments added:     +{updated_comments - original_comments}")
else:
    print("❌ No backup file available for comparison")

## Batch Process Multiple Files

In [ ]:
# Process multiple topology files
yaml_files = [
    "../model/_slice_topology.yaml",
    # Add more files here as needed
]

print("📦 Batch Processing Topology Files\n")

for yaml_file in yaml_files:
    if not Path(yaml_file).exists():
        print(f"⏭️  Skipping (not found): {yaml_file}")
        continue
    
    try:
        print(f"\n🔄 Processing: {yaml_file}")
        
        # Load topology
        topo = load_topology_from_yaml_file(yaml_file)
        
        # Inject summary
        viewer.inject_summary_into_yaml_file(
            yaml_file,
            topo,
            include_ascii=True,
            backup=True
        )
        
        print(f"   ✅ Summary injected")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")

print("\n✅ Batch processing complete!")

---

# Summary

This notebook provides two ways to generate topology summaries:

## Option 1: Command-Line Tool
- ✅ Simple one-line commands
- ✅ Easy to automate in scripts
- ✅ Good for CI/CD pipelines

## Option 2: Programmatic API
- ✅ Full control from Python/Jupyter
- ✅ Preview before modifying
- ✅ Integrate with your workflow
- ✅ Batch processing support

Both approaches:
- 🔒 Create automatic backups
- 📝 Generate detailed summaries
- 🎨 Include ASCII diagrams
- ✅ Work with model-based topologies